In [4]:
# msds607_wk9
import requests

ModuleNotFoundError: No module named 'requests'

In [2]:
import os

In [3]:
os.sys.path

['/Users/nicholas.chung/Workbench/cuny_msds/data_607/week_09',
 '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Users/nicholas.chung/Library/Python/3.7/lib/python/site-packages',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/nicholas.chung/.ipython']